In [ ]:
import pandas as pd
import numpy as np
%run similarities.ipynb
from datetime import timedelta

import os
import sys

In [ ]:
def treatment_effect_grid(yes_df, no_df, main_df, comparables, checks, check_vals, neighbors = 10):
    
    search_df = yes_df[comparables].copy()
    compare_df = no_df[comparables].copy()
    
    for idx, row in yes_df.iterrows():
        closest = find_approx_closest_euc(compare_df[no_df.date == yes_df.date[idx]].copy(), row,checks,check_vals, neighbors)
        nearest_idx[(yes_df.fips[idx], yes_df.date[idx])] = no_df.fips[closest].values
        
    low_count = 0
    for key in nearest_idx:
        low_count += len(nearest_idx[key]) < neighbors
        
    treatment_effect = []
    counterfactual_list = []
    outcome = []
    fips_list = []
    treatment_outcome = "past_week_cases"
    time_frame = 14

    for key, val in nearest_idx.items():

        check_date = key[1] + timedelta(days = time_frame)
        actual = main_df[(main_df.fips == key[0]) & (main_df.date == check_date)][treatment_outcome].values.item()
        if np.isnan(actual):
            continue

        if len(val) == 0:
            continue

        counterfactual = np.nanmean(main_df[(main_df.fips.isin(val)) & (main_df.date == check_date)][treatment_outcome].values)

        if np.isnan(counterfactual):
            continue

        fips_list.append(key[0])
        counterfactual_list.append(counterfactual)
        outcome.append(actual)
        treatment_effect.append(actual - counterfactual)

    treatment_df = pd.DataFrame({'fips': fips_list, 'effect':treatment_effect, 'actual':outcome, 'counter':counterfactual_list})

    avg_treatment = np.mean(treatment_df.effect)
    avg_treatment.sort()
    treatment_ci = (np.percentile(avg_treatment, 2.5), np.percentile(avg_treatment, 97.5))

    return [avg_treatment, treatment_ci, low_count]
